In [1]:
import cv2
import time

In [4]:
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model ='frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model,config_file)

In [6]:
classLabels = [] 
file_name='labels.txt'
with open(file_name,'rt') as fpt:
    classLabels=fpt.read().rstrip('\n').split('\n') 

In [7]:
model.setInputSize(320,320)
model.setInputScale(1.0/127.5) # 255/2 (grey level/2)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)

<dnn_Model 000002093D75EAF0>

In [8]:
from1 = False
from2 = False
count = 0

In [32]:
class CircularQueue:
  
  def __init__(self, maxSize):
    self.queue = [None for i in range(maxSize)]
    self.maxSize = maxSize
    self.head = 0
    self.tail = 0

  def enqueue(self, data):
    self.queue[self.tail] = data
    self.tail = (self.tail+1) % self.maxSize
    return
  
  def last(self):
    return self.queue[(self.tail-2)%self.maxSize]
    
  def last2(self):
    return self.queue[(self.tail-3)%self.maxSize]
  
size = 4
q = CircularQueue(int(size))
r = CircularQueue(int(size))

for i in range(size):
  q.enqueue(100000)
  r.enqueue(100000)

In [34]:
def compare():

    font_scale = 3
    font = cv2.FONT_HERSHEY_PLAIN
    cap1 = cv2.VideoCapture(0)
    cap2 = cv2.VideoCapture(1)
    while(True):
        cap1 = cv2.VideoCapture(0)
        ret,frame1 = cap1.read()
        ClassIndex1, confidece1, bbox1= model.detect(frame1, confThreshold= 0.55)
    
        if (len(ClassIndex1)!=0):
            if(ClassIndex1[0]==1):
                area = (bbox1[0][2]-bbox1[0][0])*(bbox1[0][3]-bbox1[0][1])
                q.enqueue(area)
                if(area > q.last() and area > q.last2()):
                    from1 = True
                else:
                    from1 = False
                print(area , q.last() ,q.last2() )
        else:
            from1 = False

        # cv2.imshow('Object Detection Tutorial',frame1)
        
        cap2 = cv2.VideoCapture(1)
        ret,frame2 = cap2.read()
        ClassIndex2, confidece2, bbox2= model.detect(frame2, confThreshold= 0.55)
    
        if (len(ClassIndex2)!=0):
            if(ClassIndex2[0]==1):
                area = (bbox2[0][2]-bbox2[0][0])*(bbox2[0][3]-bbox2[0][1])
                r.enqueue(area)
                if(area > r.last() and area > r.last2()):
                    from2 = True
                else:
                    from2 = False
                print(area , r.last() , r.last2())
        else:
            from2 = False

        # cv2.imshow('Object Detection Tutorial',frame2)

        if(from1==True and from2==True):
            print("Red")
            time.sleep(0.5)
        else:
            print("Green")
        time.sleep(0.5) 
        
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break


    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

In [35]:
compare()

168980 100000 100000
226980 100000 100000
Red
14154 168980 100000
-10304 226980 100000
Green
63855 14154 168980
89760 -10304 226980
Green
21372 63855 14154
-11040 89760 -10304
Green
152650 21372 63855
160270 -11040 89760
Red
155100 152650 21372
218530 160270 -11040
Red
211864 155100 152650
291711 218530 160270
Red
156200 211864 155100
266655 291711 218530
Red
51840 156200 211864
266655 266655 291711
Green
193185 51840 156200
203904 266655 266655
Green
-1358 193185 51840
1656 203904 266655
Green
56304 -1358 193185
65720 1656 203904
Green
71200 56304 -1358
118592 65720 1656
Green
127007 71200 56304
203688 118592 65720
Red
76560 127007 71200
223200 203688 118592
Red


KeyboardInterrupt: 